# 为 Shapefile 添加随机属性字段

本工具用于**为 Shapefile 批量添加模拟的管线属性字段**，包括高程、管径、节点ID、管材类型等，常用于测试数据生成和演示。

## 主要功能

1. **打断线段**：将多段线（Polyline）打断为直线段（两点间的 LineString）
2. **添加随机属性**：
   - 起始/终点高程（`us_invert`, `ds_invert`）
   - 管径（`condwidth`）
   - 节点 ID（`us_node_id`, `ds_node_id`）
   - 管材类型（`gdcz`）
   - 资产 ID（`asset_id`）

## 应用场景

- 生成测试数据用于管网分析
- 为缺失属性的管线数据填充模拟值
- 演示和培训用数据准备
- 水力模型输入数据准备

> **依赖库**  
> - GeoPandas
> - Pandas
> - NumPy
> - Shapely
> 
> 安装命令：  
> ```bash
> pip install geopandas pandas numpy shapely
> ```

In [4]:
import geopandas as gpd
from shapely.geometry import LineString

# 读取原始道路 Shapefile
input_path = "E:/项目管理/深泽/深泽/数据/ROADS/roads.shp"
gdf = gpd.read_file(input_path)

# 创建新数据列表
new_rows = []

for idx, row in gdf.iterrows():
    line: LineString = row.geometry
    
    # 确保是 LineString
    if line and line.geom_type == "LineString":
        coords = list(line.coords)
        
        # 将每两个点之间的线段作为新 LineString
        for i in range(len(coords) - 1):
            new_line = LineString([coords[i], coords[i+1]])
            
            # 保留原有属性（可选）
            new_row = row.copy()
            new_row.geometry = new_line
            new_rows.append(new_row)

# 创建新的 GeoDataFrame
new_gdf = gpd.GeoDataFrame(new_rows, crs=gdf.crs)

# 保存打断后的结果
output_path = "E:/项目管理/深泽/深泽/数据/ROADS/broken_lines.shp"
new_gdf.to_file(output_path, encoding="utf-8")

print(f"线段打断完成，共生成 {len(new_gdf)} 条直线段。保存路径：{output_path}")


线段打断完成，共生成 8842 条直线段。保存路径：E:/项目管理/深泽/深泽/数据/ROADS/broken_lines.shp


## 步骤 1: 将多段线打断为单线段

将复杂的 Polyline 分解为两点间的简单 LineString。

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random

# 加载原始 shp 文件
shp_path = "E:/项目管理/深泽/深泽/数据/ROADS/broken_lines.shp"  # TODO: 修改为你的文件路径
gdf = gpd.read_file(shp_path)

# 设置记录总数
n = len(gdf)

# 添加字段并生成值
gdf["us_invert"] = np.round(np.random.uniform(10, 40.0, size=n), 2)  # 起始高程
gdf["ds_invert"] = np.round(np.random.uniform(10, 40.0, size=n), 2)  # 终点高程
gdf["condwidth"] = np.random.choice([200, 300, 400, 500, 600], size=n)  # 管径（mm）

# 节点 ID
gdf["us_node_id"] = ["US{:04d}".format(i+1) for i in range(n)]
gdf["ds_node_id"] = ["DS{:04d}".format(i+1) for i in range(n)]

# 管道材质类型
pipe_types = ["钢管", "铸铁管", "聚乙烯", "聚丙烯", "PVC"]
gdf["gdcz"] = np.random.choice(pipe_types, size=n)

# 管道唯一标识
gdf["asset_id"] = ["PIPE{:05d}".format(i+1) for i in range(n)]

# 保存为新的 shp 文件
output_path = "E:/项目管理/深泽/深泽/数据/ROADS/pipeline.shp"
gdf.to_file(output_path, encoding="utf-8")

print(f"新字段添加完成并保存至：{output_path}")


新字段添加完成并保存至：E:/项目管理/深泽/深泽/数据/ROADS/pipeline.shp


## 步骤 2: 添加随机管线属性

为每条线段生成模拟的管网属性数据。